## Multi-Modal RAG with Azure AI Content Understanding - In Action

![rag_in_action](./Assets/rag_in_action.png)

In [ ]:
%pip install python-dotenv openai

### Setting up the Environment

In [ ]:
import os
from dotenv import load_dotenv
import requests
import json

load_dotenv()

# Azure OpenAI Configurations
azure_ai_endpoint = os.getenv("AZURE_AI_ENDPOINT")
azure_ai_api_key = os.getenv("AZURE_AI_API_KEY")
embedding_model_name = os.getenv("EMBEDDING_MODEL_NAME")
chat_model_name = os.getenv("CHAT_MODEL_NAME")

# Azure Search Service Configurations
search_service_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT")
search_service_api_key = os.getenv("SEARCH_SERVICE_API_KEY")
search_service_index_name = os.getenv("SEARCH_SERVICE_INDEX_NAME")



### Creating Azure OpenAI Client

In [ ]:
from openai import AzureOpenAI

openai_client = AzureOpenAI(
    api_key = os.getenv("AZURE_AI_API_KEY"),
    api_version = "2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_AI_ENDPOINT")
)

### Creating a Function for Generating Vector Embeddings using Azure OpenAI

In [ ]:
from openai import AzureOpenAI

def generate_embeddings(text, embedding_model_name):
    response = openai_client.embeddings.create(
        input=text,
        model=embedding_model_name
    )

    embeddings = response.model_dump()

    return embeddings["data"][0]["embedding"]

In [ ]:
"""
Queries to try out:
1. tell me something about BMW Circularity Project
2. Tell me something about BMW's forwardism strategy
3. Tell me something about BMW's sustainable natural rubber initiative
4. Tell me through BMW's sustainability journey from 1973 to 2030
"""

user_query = "tell me something about BMW Circularity Project"
vectorised_user_query = generate_embeddings(user_query, embedding_model_name)
print(vectorised_user_query)

In [ ]:
context=[]

### Sending API call to the Search Index to Build Context

In [ ]:
import requests
import json


url = f"{search_service_endpoint}/indexes/{search_service_index_name}/docs/search?api-version=2023-11-01"
    
headers = {
        "Content-Type": "application/json",
        "api-key": search_service_api_key
    }
    
body =   {
        "count": True,
        "select": "content_text",
        "vectorQueries": [
            {
                "vector": vectorised_user_query,
                "k": 10,
                "fields": "content_embedding",
                "kind": "vector"
            }
        ]
    }
    
response = requests.post(url, headers=headers, data=json.dumps(body))
documents = response.json()['value']

for doc in documents:
    context.append(dict(
        {
            "chunk": doc['content_text'],
            "score": doc['@search.score']
            
        }
    ))
    
for doc in context:
    print(doc)

### Setting the System Prompt

In [ ]:
system_prompt = """
You are a professional RAG-based assistant whose context comes exclusively from a database in Azure AI Search

Always follow these rules:

1. Answer strictly from the context provided. Do not invent, assume, or add details outside the given context.
2. If no relevant information is available in the context, politely say so.
3. Do not include any external links, citations, or references unless they are explicitly present in the context object.
4. Context format: The system will pass a Python list of objects, each containing:
   {
     "chunk": "the content (text, JSON, transcript, or description)",
     "score": "the relevancy score"
   }
   These are the top matches based on cosine similarity with the user query.
5. Style & tone:
   - Respond in a professional, natural way, as if conversing with a human.
   - Structure answers clearly and concisely.
   - Do not reveal that retrieval-augmented generation (RAG) is being used under the hood.
   - If the context contains a field such as 'url', 'video_url', or 'image_url', include it as a clickable hyperlink in your response, but do not fabricate or assume URLs.

Your role is to act like a knowledgeable human assistant who can reference the provided information smoothly and contextually, across any modality (text, image, video, or review description).
"""


### Calling GPT Engine for Summarisation

In [ ]:
user_query = f""" The user query is: {user_query}
              the context is: {context} """

chat_completions_response = openai_client.chat.completions.create(
    model = chat_model_name,
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_query}
    ],
    temperature=0.7
)

print(chat_completions_response.choices[0].message.content)